Check this out for inspiration:
https://gist.github.com/mandyRae/459ae289cdfcf6d98a6b

In [1]:
'''
Written  by: Jeffrey Blitt
Modified on: 01/11/19
'''

# Import all necessary libraries for the program to run
import RPi.GPIO as GPIO  # Regular pin input/output package for Pi
from time import sleep   # Sleep function for musical timing
from lxml import html    # Retrieval of HTML from a URL
import re                # Regular Expression package
import requests          # For web content decryption
GPIO.setwarnings(False)  # Ignore warnings
GPIO.setmode(GPIO.BCM)   # Set pin reference mode to BCM

# Constants
BUZZ1PIN = 2             # Declare the pin used for output to the buzzer
BEAT = .125              # Declare the length of one beat (really 4/5 of a beat)

# Declare the frequency of each note in octave zero (using A 440 tuning)
noteDict = {
    "C" : 16.35,
    "C#": 17.32,
    "Db": 17.32,
    "D" : 18.35,
    "D#": 19.45,
    "Eb": 19.45,
    "E" : 20.60,
    "F" : 21.83,
    "F#": 23.12,
    "Gb": 23.12,
    "G" : 24.50,
    "G#": 25.96,
    "Ab": 25.96,
    "A" : 27.50,
    "A#": 29.14,
    "Bb": 29.14,
    "B" : 30.87,
    ""  :     0
}

# Declare the characters allowed in a note string
allowedChars = ['A','B','C','D','E','F','G','b','#',' ','_','.','^','*']

In [2]:
def note_to_freq(note):
    if note == "":
        octave = 0
        freq   = 0
    elif str(note)[-1].isdigit() == True:
        octave = int(str(note)[-1])  # take last char in note to be octave num
        note   = note[:-1]
        freq   = float(noteDict.get(note)) * (2 ** octave)  # get the corresponding frequency
    else:
        octave = 4
        freq   = float(noteDict.get(note)) * (2 ** octave)  # get the corresponding frequency
    return freq

def get_html(url):
    page = requests.get(url)
    html = page.content.decode("utf-8")
    return html

def strip_tags(html):
    html = re.sub('<[^<]+?>', '', html)
    return html

def parse_html_text(html):
    html = str(html)
    rawScraped = html.split('<div class="post-content">\r\n')[1]
    cleanMusic = strip_tags(rawScraped)
    cleanMusic = cleanMusic.replace('\xa0', '')
    cleanMusic = cleanMusic.replace('&nbsp;', '_')
    cleanMusic = cleanMusic.replace('-', ' ')
    cleanMusic = cleanMusic.lstrip()
    songTextArray = cleanMusic.split('\n')
    notesList = []
    for line in songTextArray:
        if line[0] == ' ':
            break
        else:
            while '  ' in line:
                line = line.replace('  ', ' ')
            lineLength = len(line)
            for charIndex in range(lineLength):
                if line[charIndex] in allowedChars:
                    if charIndex == lineLength - 1:
                        notesList.append(line)
                    else: continue
                else: break
    return notesList
    
class Buzzer:
    # Usage of constructor - "buzzerName = Buzzer(<pin>)"
    def __init__(self, buzzerPin):
        self.pin = buzzerPin
        GPIO.setup(self.pin, GPIO.IN)
        GPIO.setup(self.pin, GPIO.OUT)
    def __del__(self):                             # IDK how these destructors work
        class_name = self.__class__.__name__
        GPIO.cleanup
            
    def stop_tone(self):
        GPIO.PWM(self.pin, 0.1).stop()
            
    def play_tone(self, freq):
        if freq == 0:
            sleep(BEAT)
        else:
            tone = GPIO.PWM(self.pin, freq)
            tone.start(50)
            sleep(BEAT)
            tone.stop()
        
    def play_note(self, note):
        freq = note_to_freq(note)
        self.play_tone(freq)
        
    def play_song(self, songArray):
        for line in songArray:
            line = line.split(' ')
            print(line)
            for rawNote in line:
                note = "C"
                octave = 4
                if rawNote == '_':
                    sleep(BEAT)
                    continue
                elif rawNote == '':
                    continue
                elif rawNote[0] == '^':
                    octave += 1
                    note = rawNote[1:]
                elif rawNote[0] == '*':
                    octave += 2
                    note = rawNote[1:]
                elif rawNote[0] == '.':
                    octave -= 1
                    note = rawNote[1:]
                elif rawNote[0] == '_':
                    octave -= 2
                    note = rawNote[1:]
                else: note = rawNote
                note = note + str(octave)
                self.play_note(note)
                sleep(BEAT / 2)
            sleep(BEAT * 1.5)

In [ ]:
# Manual function testing
songs = ['https://noobnotes.net/how-to-save-a-life-the-fray/?solfege=false',
         'https://noobnotes.net/super-mario-bros-theme-nintendo/?solfege=false',
         'https://noobnotes.net/happy-birthday-traditional/?solfege=false',
         'https://noobnotes.net/bohemian-rhapsody-queen/?solfege=false&transpose=0',
         'https://noobnotes.net/let-it-go-frozen-disney/?solfege=false']
buzzer = Buzzer(BUZZ1PIN)
rawHtml = get_html(songs[3])

musicArray = parse_html_text(rawHtml)
try:
    buzzer.play_song(musicArray)
except KeyboardInterrupt:
    buzzer.stop_tone()